#Klasyfikacja zbiorów: MNIST, CIFAR-10, CIFAR-100


In [ ]:
from keras.datasets import mnist
from keras.datasets import cifar10

from keras import models
from keras import layers
from keras.utils import to_categorical

# MNIST

**Przygotowanie danych**

In [ ]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
print('Train dataset')
print(train_images.shape)
print(len(train_labels))
print(train_labels)
print('Test dataset')
print(test_images.shape)
print(len(test_labels))
print(test_labels)

11490434/11490434 [==============================] - 0s 0us/step
Train dataset
(60000, 28, 28)
60000
[5 0 4 ... 5 6 8]
Test dataset
(10000, 28, 28)
10000
[7 2 1 ... 4 5 6]




*   Zbiór MNIST składa się z obrazów ręcznie zapisanych cyfr o 10 kategoriach 
(cyfry 0-9)
*   Zbiór treningowy składa się z 60000 obrazów o rozmiarze 28x28 
*   Zbiór testowy składa się z 10000 obrazów o rozmiarze 28x28 


Przygotowanie danych w postaci obrazów 

In [ ]:
train_images = train_images.reshape((60000, 28*28))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28*28))
test_images = test_images.astype('float32') / 255

* Przeskalowanie pikseli z [0-255] do [0-1]
* Zmiana z typu uint8 na float32
* Wyjście w postaci 1 wymiarowego tensora: [60000, 28 28], [10000, 28 28]

Przygotowanie etykiet

In [ ]:
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

Zamiana etykiet z wartości [0, 9] na 10 wymiarowy wektor który ma reprezentować cyfrę w postaci 'binarnej'

**Tworzenie sieci**

In [ ]:
network = models.Sequential()
network.add(layers.Dense(512, activation='relu', input_shape=(28*28,)))
network.add(layers.Dense(10, activation='softmax'))

* Sequential - model sieci z sekwencją warstw, z których każda stopniowo ekstraktuje dane w celu uzyskania pożądanego wyniku
* Dense - sieci połączone gęsto
* Relu - funkcja nieliniowa, dodatnie wejścia przechodzą bez zmian, jednocześnie ograniczając wszystko inne do zera
* Softmax - zamienia wektor liczb na wektor prawdopodobieństw
* Druga warstwa jest 10-elementową wartstwą zawracającą tablicę 10 wartości prawdopodbieństwa (których suma = 1), oznacza to prawdopodbienstwo wystapienia cyfry na obrazie

Kompilacja sieci

In [ ]:
network.compile(optimizer='rmsprop',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

*   Optymalizator - rmsprop   
*   Funkcja straty - 
*   Metryka - accuracy - liczba poprawnie zaklasyfikowanych obrazów



Trenowanie sieci

In [ ]:
network.fit(train_images, train_labels, epochs=5, batch_size=128)

Epoch 1/5
469/469 [==============================] - 3s 6ms/step - loss: 0.2642 - accuracy: 0.9244
Epoch 2/5
469/469 [==============================] - 3s 6ms/step - loss: 0.1071 - accuracy: 0.9691
Epoch 3/5
469/469 [==============================] - 3s 6ms/step - loss: 0.0698 - accuracy: 0.9793
Epoch 4/5
469/469 [==============================] - 3s 7ms/step - loss: 0.0510 - accuracy: 0.9850
Epoch 5/5
469/469 [==============================] - 3s 6ms/step - loss: 0.0374 - accuracy: 0.9887


In [ ]:
test_loss, test_acc = network.evaluate(test_images, test_labels)
print('test_loss: ', test_loss)
print('test_acc: ', test_acc*100)

313/313 [==============================] - 1s 2ms/step - loss: 0.0694 - accuracy: 0.9780
test_loss:  0.06939179450273514
test_acc:  97.79999852180481


# CIFAR-10

**Przygotowanie danych**

In [ ]:
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()
print('Train dataset')
print(train_images.shape)
print(len(train_labels))
print(train_labels.shape)
print('Test dataset')
print(test_images.shape)
print(len(test_labels))
print(test_labels.shape)

170498071/170498071 [==============================] - 6s 0us/step
Train dataset
(50000, 32, 32, 3)
50000
(50000, 1)
Test dataset
(10000, 32, 32, 3)
10000
(10000, 1)


*   Zbiór CIFAR-10 składa się z  60000 kolorowych obrazów o wymiarach 32×32 w 10 klasach
*   Zbiór treningowy składa się z 50000 obrazów
*   Zbiór testowy składa się z 10000 obrazów

Przygotowanie danych w postaci obrazów

In [ ]:
train_images = train_images.astype('float32') / 255
test_images = test_images.astype('float32') / 255

Przygotowanie etykiet

In [ ]:
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

Tworzenie sieci

In [ ]:
network = models.Sequential()

network.add(layers.Conv2D(32, (3,3), padding='same', activation='relu', input_shape=(32,32,3)))
network.add(layers.BatchNormalization())
network.add(layers.Conv2D(32, (3,3), padding='same', activation='relu'))
network.add(layers.BatchNormalization())
network.add(layers.MaxPooling2D(pool_size=(2,2)))
network.add(layers.Dropout(0.3))

network.add(layers.Conv2D(64, (3,3), padding='same', activation='relu'))
network.add(layers.BatchNormalization())
network.add(layers.Conv2D(64, (3,3), padding='same', activation='relu'))
network.add(layers.BatchNormalization())
network.add(layers.MaxPooling2D(pool_size=(2,2)))
network.add(layers.Dropout(0.5))

network.add(layers.Conv2D(128, (3,3), padding='same', activation='relu'))
network.add(layers.BatchNormalization())
network.add(layers.Conv2D(128, (3,3), padding='same', activation='relu'))
network.add(layers.BatchNormalization())
network.add(layers.MaxPooling2D(pool_size=(2,2)))
network.add(layers.Dropout(0.5))

network.add(layers.Flatten())
network.add(layers.Dense(128, activation='relu'))
network.add(layers.BatchNormalization())
network.add(layers.Dropout(0.5))
network.add(layers.Dense(10, activation='softmax'))    # num_classes = 10


Kompilacja sieci

In [ ]:
network.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

Trenowanie sieci

In [ ]:
history = network.fit(train_images, train_labels, batch_size=64, epochs=10,
                    validation_data=(test_images, test_labels))

Epoch 1/10
782/782 [==============================] - 281s 360ms/step - loss: 1.6306 - accuracy: 0.4235 - val_loss: 1.2409 - val_accuracy: 0.5543
Epoch 2/10
782/782 [==============================] - 286s 366ms/step - loss: 1.1571 - accuracy: 0.5856 - val_loss: 0.9430 - val_accuracy: 0.6706
Epoch 3/10
782/782 [==============================] - 283s 362ms/step - loss: 0.9754 - accuracy: 0.6563 - val_loss: 1.1149 - val_accuracy: 0.6189
Epoch 4/10
782/782 [==============================] - 283s 363ms/step - loss: 0.8765 - accuracy: 0.6925 - val_loss: 0.8183 - val_accuracy: 0.7181
Epoch 5/10
782/782 [==============================] - 284s 363ms/step - loss: 0.8067 - accuracy: 0.7202 - val_loss: 0.6987 - val_accuracy: 0.7598
Epoch 6/10
782/782 [==============================] - 291s 372ms/step - loss: 0.7555 - accuracy: 0.7388 - val_loss: 0.8327 - val_accuracy: 0.7127
Epoch 7/10
782/782 [==============================] - 284s 363ms/step - loss: 0.7037 - accuracy: 0.7570 - val_loss: 0.6100 -

In [ ]:
test_loss, test_acc = network.evaluate(test_images, test_labels)
print('test_loss: ', test_loss)
print('test_acc: ', test_acc*100)

313/313 [==============================] - 14s 43ms/step - loss: 0.6428 - accuracy: 0.7859
test_loss:  0.6428285837173462
test_acc:  78.58999967575073
